In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/PhanLoaiAmThanhDongVat

/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [ ]:
import pandas as pd
import torch
import copy
import time
from pathlib import Path
from torch import nn
import tensorflow as tf
import torchvision.transforms as transforms
import torchvision.models as models
import torchaudio
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from audio_util_load import AudioUtil
from sound_classification_dataset import SoundDS
import sklearn
from sklearn.model_selection import KFold
import numpy as np
import gc


In [ ]:
import os

In [ ]:

# Read metadata file
metadata_file = '/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat/animalnewlabelled.csv'
df = pd.read_csv(metadata_file)
df.head()

,Filename,Label,ClassID,Path
0,Dolphin_17.wav,Dolphin,6,/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat...
1,Dolphin_20.wav,Dolphin,6,/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat...
2,Dolphin_16.wav,Dolphin,6,/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat...
3,Dolphin_13.wav,Dolphin,6,/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat...
4,Dolphin_39.wav,Dolphin,6,/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat...


In [ ]:
df.shape

(2177, 4)

In [ ]:
writer = SummaryWriter()
myds = SoundDS(df)
# For fold results
results = {}

In [ ]:
# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=64, shuffle=False)

In [ ]:
# Load the pre-trained ResNet-50 model
model = models.densenet161(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
               nn.Linear(2208, 128),
               nn.ReLU(inplace=True),
               nn.Dropout(0.2),
               nn.Linear(128, 13),

               )

model = model.to(device)
# Check that it is on Cuda
next(model.parameters()).device

device(type='cpu')

In [ ]:
# ----------------------------
# Inference
# ----------------------------
def inference (model, test_dl):
    correct_prediction = 0
    total_prediction = 0

    # Disable gradient updates
    with torch.no_grad():
        for data in test_dl:
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

    acc = correct_prediction/total_prediction
    print(f'Test Accuracy: {acc:.2f}, Total items: {total_prediction}')

In [ ]:
def training(model, train_dl, num_epochs):
    # Loss Function, Optimizer and Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

    # Repeat for each epoch
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

        # Repeat for each batch in the training set
        for i, data in enumerate(train_dl):
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Keep stats for Loss and Accuracy
            running_loss += loss.item()

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]
            #if i % 10 == 0:    # print every 10 mini-batches
            #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
            del inputs_m
            del inputs_s
            del inputs,labels
            gc.collect()
        # Print stats at the end of the epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        avg_acc = correct_prediction/total_prediction
        writer.add_scalar("Loss/train", avg_loss, epoch)
        writer.add_scalar("Acc/train", avg_acc, epoch)
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {avg_acc:.2f}')

    print('Finished Training')


In [ ]:
num_epochs=50
training(model, train_dl, num_epochs)

Epoch: 0, Loss: 2.53, Accuracy: 0.12


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat/densenet161foranimalNew.pt')

In [ ]:
# Run inference on trained model with the validation set load best model weights
model_inf =models.densenet161(pretrained=True)
for param in model_inf.parameters():
    param.requires_grad = False
model_inf.classifier= nn.Sequential(
               nn.Linear(2208, 128),
               nn.ReLU(inplace=True),
               nn.Dropout(0.2),
               nn.Linear(128, 13),

               )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_inf = model_inf.to(device)
model_inf.load_state_dict(torch.load('/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat/densenet161foranimalNew.pt'))
model_inf.eval()
inference(model_inf, val_dl)

<ipython-input-34-7a63ef846cf9>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_inf.load_state_dict(torch.load('/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat/densen

Test Accuracy: 0.82, Total items: 435


In [ ]:
# Load the saved state dictionary
state_dict = torch.load('/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat/densenet161foranimalNew.pt')

# Create an instance of your model
model = models.densenet161(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Sequential(
               nn.Linear(2208, 128),
               nn.ReLU(inplace=True),
               nn.Dropout(0.2),
               nn.Linear(128, 13),
               )
# Load the state dictionary into the model
model.load_state_dict(state_dict)
model.eval()
inputs = AudioUtil.open('/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat/DuLieuKiemTra/lion-roar-sound-effect.wav')
inputs = AudioUtil.rechannel(inputs,2)
inputs=AudioUtil.resample(inputs,44100)
inputs=AudioUtil.pad_trunc(inputs,4000)
inputs = AudioUtil.spectro_gram(inputs, n_mels=224, n_fft=4096, hop_len=None)

inputs = F.interpolate(inputs.unsqueeze(0), size=(224, 224), mode='bilinear').squeeze(0)
class_labels=[]
class_labels=["sư tử","gấu","Mèo","gà","bò","chó","cá heo","lừa","voi","ếch","ngựa","khỉ","cừu"]

        # Convert 2 channels to 3 channels by duplicating one of the channels
inputs = torch.cat([inputs, inputs[:1, :, :]], dim=0)
    # Make prediction using the model
with torch.no_grad():
     inputs=inputs.unsqueeze(0).to("cpu")
     inputs_m, inputs_s = inputs.mean(), inputs.std()
     inputs = (inputs - inputs_m) / inputs_s
     outputs = model(inputs)
     _, prediction = torch.max(outputs, 1)
rs=prediction[0].item()
print(f'Dự đoán nhãn : {rs}')
print("có thể là con : "+class_labels[rs])

<ipython-input-28-825a2c82f727>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/content/gdrive/MyDrive/PhanLoaiAmThanhDongVat/densenet161foranimal

Dự đoán nhãn : 0
có thể là con : sư tử


In [ ]:
cp /content/dolphin4_09.wav /content/gdrive/MyDrive/PhanLoaiAmThanhDongVat/DuLieuKiemTra/dolphin4_09.wav